In [1]:
import pickle

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

# load the complete text-id dictionary of queries (pre-constructed)
question_id = pickle.load(open('data/all_questions_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('data/all_questions_title.pkl','rb'))

# load the complete url-id dictionary of pages (pre-constructed)
page_id = pickle.load(open('data/infowave_allintents_dict.pkl','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))

# question-intent features (pre-constructed)
question_intent_features = pickle.load(open('data/all_questions_intent.pkl','rb'))



def sample_by_page_id():
    # sample the data by page-id, here we limit a number of page ids,  only keep the page < 10
    sample_threhold = 10

    ratings=[]
    dict_question={}
    dict_question_embed={}
    dict_question_intent={}
    dict_q_mapping={}
#     dict_url={}
#     dict_title={}

    for x,y in association_mat:
        if (y<sample_threhold): # filter by page id
            ratings.append((x,y))

            #print(x,y)

    all_q_ids = [x for (x,y) in ratings]
    all_i_ids = [y for (x,y) in ratings]

    icount = 0

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1

    
    new_ratings=[]

    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))

    ratings = new_ratings

    dict_question={}
    dict_question_embed={}

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=dict_q_mapping[v]
            dict_question_embed[k]=question_features[k]
            #dict_question_intent[k]=question_intent_features[k]

    pickle.dump(ratings,open('data/sample_by_page_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_question,open('data/sample_by_page_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_page_questions_features.pkl','wb'))
    #pickle.dump(dict_question_intent,open('data/sample_by_page_questions_intent_features.pkl','wb'))

    
sample_by_page_id()

In [40]:
import pickle

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

# load the complete text-id dictionary of queries (pre-constructed)
question_id = pickle.load(open('data/all_questions_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('data/all_questions_title.pkl','rb'))

# load the complete url-id dictionary of pages (pre-constructed)
page_id = pickle.load(open('data/infowave_allintents_dict.pkl','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))

# question-intent features (pre-constructed)
question_intent_features = pickle.load(open('data/all_questions_intent.pkl','rb'))

In [41]:
import random 
random.sample
sample_list = random.sample(range(995),300)

def sample_by_query_id(sample_list):
    # sample the data by query-id, 
    # sample_threhold = 900
    
    ratings = []
    
    #dict_i_mapping={}  # for hold out validation setting, we should not shrink the page id
    
    dict_q_mapping={}  

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}  
    
    dict_question={}
    dict_question_embed={}
    dict_question_intents={}
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        #if (x<sample_threhold): # filter by query id
        if (x in sample_list): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings]
    #all_i_ids = [y for (x,y) in ratings]            
    
    #print(all_i_ids)
    
    icount = 0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1    

    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        #if ((v[0] in all_i_ids) and (k in page_features)):
        if k in page_features:                
            #print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
#             if v[0] not in dict_i_mapping:
#                 dict_i_mapping[v[0]]=icount
#                 icount+=1
                
#   print(dict_q_mapping)
   
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))
    
    
    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

    dict_question={}
    dict_question_embed={}
    #dict_q_mapping={}

    for k,v in page_id.items():
        #if (v[0] in all_i_ids) and (k in page_features):
        if k in page_features:

            dict_url[k]=v[0]
            dict_url_embed[k]=page_features[k]    
            
    for q_id in all_q_ids:
        #print(dict_q_mapping[q_id])
        #print(q_id)
        dict_question_intents[dict_q_mapping[q_id]]=question_intent_features[q_id]            
    
    #print(dict_i_mapping)
    dict_q_mapping={}
    icount =0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=icount
            dict_question_embed[k]=question_features[k]
            
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1
                
    pickle.dump(dict_question,open('data/sample_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_question_questions_features.pkl','wb'))            
    pickle.dump(ratings,open('data/sample_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/sample_by_question_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/sample_by_question_page_features.pkl','wb'))
    pickle.dump(dict_question_intents,open('data/sample_by_question_query_intents.pkl','wb'))

sample_by_query_id(sample_list) 

In [24]:
# save the hold out data
def holdout_by_query_id(sample_list):
    # sample the data by query-id, 
    # sample_threhold = 900

    ratings = []
    #dict_i_mapping={}
    
    dict_q_mapping={}

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}  
    
    dict_question={}
    dict_question_embed={}
    dict_question_intents={}
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        #if (x<sample_threhold): # filter by query id
        if (x not in sample_list): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings]
    #all_i_ids = [y for (x,y) in ratings]            
    
    #print(all_i_ids)
    
    icount = 0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1    

    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        #if ((v[0] in all_i_ids) and (k in page_features)):
        if k in page_features:
            #print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
#             if v[0] not in dict_i_mapping:
#                 dict_i_mapping[v[0]]=icount
#                 icount+=1
                
    print(dict_q_mapping)
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        #new_ratings.append((dict_q_mapping[x],dict_i_mapping[y]))
        new_ratings.append((dict_q_mapping[x],y))

    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

#     dict_question={}
#     dict_question_embed={}

    for k,v in page_id.items():
        #if (v[0] in all_i_ids) and (k in page_features):
        if k in page_features:
            dict_url[k]=v[0]
            dict_url_embed[k]=page_features[k]    
            
    for q_id in all_q_ids:
        #print(dict_q_mapping[q_id])
        #print(q_id)
        dict_question_intents[dict_q_mapping[q_id]]=question_intent_features[q_id]            
    
    #print(dict_i_mapping)
    
    icount =0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=icount
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1
                
    pickle.dump(dict_question,open('data/heldout_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/heldout_by_question_questions_features.pkl','wb'))            
    pickle.dump(ratings,open('data/heldout_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/heldout_by_question_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/heldout_by_question_page_features.pkl','wb'))
    pickle.dump(dict_question_intents,open('data/heldout_by_question_query_intents.pkl','wb'))
    
    
                                          
holdout_by_query_id(sample_list)                                           

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 6, 9: 7, 10: 8, 12: 9, 13: 10, 14: 11, 16: 12, 17: 13, 18: 14, 20: 15, 21: 16, 22: 17, 23: 18, 24: 19, 25: 20, 28: 21, 29: 22, 30: 23, 31: 24, 32: 25, 33: 26, 34: 27, 36: 28, 37: 29, 38: 30, 39: 31, 40: 32, 42: 33, 43: 34, 45: 35, 46: 36, 48: 37, 49: 38, 50: 39, 51: 40, 52: 41, 54: 42, 55: 43, 56: 44, 58: 45, 59: 46, 60: 47, 61: 48, 62: 49, 63: 50, 65: 51, 67: 52, 70: 53, 72: 54, 73: 55, 75: 56, 76: 57, 77: 58, 78: 59, 79: 60, 81: 61, 82: 62, 83: 63, 84: 64, 87: 65, 88: 66, 89: 67, 91: 68, 92: 69, 93: 70, 94: 71, 98: 72, 100: 73, 103: 74, 106: 75, 107: 76, 108: 77, 109: 78, 112: 79, 113: 80, 114: 81, 116: 82, 117: 83, 118: 84, 121: 85, 122: 86, 123: 87, 125: 88, 127: 89, 128: 90, 129: 91, 131: 92, 132: 93, 133: 94, 134: 95, 136: 96, 138: 97, 139: 98, 140: 99, 141: 100, 144: 101, 146: 102, 147: 103, 150: 104, 151: 105, 152: 106, 154: 107, 155: 108, 159: 109, 162: 110, 163: 111, 164: 112, 165: 113, 166: 114, 167: 115, 168: 116, 169: 117, 170: 118, 

In [42]:
ratings = pickle.load(open('./data/sample_by_question_questions_article_ratings.pkl','rb'))
question_id = pickle.load(open('data/sample_by_question_questions_dict.pkl','rb'))
question_features = pickle.load(open('data/sample_by_question_questions_features.pkl','rb'))
question_intent_features = pickle.load(open('data/sample_by_question_query_intents.pkl','rb'))
dict_url = pickle.load(open('data/sample_by_question_page_dict.pkl','rb'))
dict_title_embed=pickle.load(open('data/sample_by_question_page_features.pkl','rb'))


In [46]:
question_intent_features

{0: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 1: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 2: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 3: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 4: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 5: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 6: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 7: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 8: array([1., 0., 0., 0., 0., 0

In [46]:
import numpy as np
#intent_features = np.empty(shape=(2,), dtype=object)
intent_features[1]=question_intent_features[1] 


[993,
 607,
 709,
 455,
 421,
 906,
 428,
 451,
 269,
 946,
 12,
 837,
 627,
 457,
 846,
 521,
 30,
 67,
 940,
 411,
 315,
 932,
 323,
 195,
 775,
 945,
 173,
 628,
 816,
 192,
 65,
 866,
 586,
 131,
 714,
 437,
 230,
 73,
 157,
 412,
 146,
 682,
 382,
 255,
 246,
 785,
 556,
 404,
 815,
 888,
 512,
 519,
 505,
 434,
 343,
 482,
 136,
 92,
 376,
 122,
 591,
 485,
 284,
 915,
 341,
 687,
 518,
 697,
 874,
 598,
 300,
 197,
 456,
 648,
 529,
 492,
 5,
 72,
 596,
 462,
 886,
 328,
 34,
 344,
 130,
 182,
 544,
 773,
 29,
 674,
 127,
 622,
 893,
 696,
 351,
 372,
 580,
 897,
 86,
 66,
 26,
 568,
 14,
 797,
 561,
 661,
 58,
 752,
 504,
 59,
 589,
 822,
 319,
 632,
 481,
 727,
 202,
 792,
 179,
 239,
 494,
 285,
 420,
 704,
 651,
 280,
 891,
 836,
 890,
 533,
 823,
 368,
 878,
 639,
 348,
 359,
 144,
 203,
 895,
 487,
 210,
 330,
 317,
 110,
 865,
 646,
 81,
 99,
 902,
 951,
 522,
 672,
 909,
 447,
 582,
 352,
 531,
 538,
 142,
 594,
 503,
 75,
 408,
 713,
 48,
 267,
 626,
 576,
 96,
 287,
 5